In [22]:
import geopandas as gpd
import pandas as pd
import numpy as np
from datetime import datetime
from rasterstats import zonal_stats
from dbfread import DBF
import re


In [23]:
# -----------------------
# Inputs and Load
# -----------------------
project_db   = r"C:\Users\bsf31\Documents\data\NL060\fire_scar_training_regions.gpkg"
dbf_path    = r"C:\Users\bsf31\Documents\data\NL060\landfire_meszxc7dfpgmqh\LF2024_FBFM40_250_CONUS\LF24_F40_250.dbf"
raster_path  = r"C:\Users\bsf31\Documents\data\NL060\landfire_meszxc7dfpgmqh\LF2024_FBFM40_250_CONUS\LC24_F40_250.tif"

Scar_layer = "canyon2016_5070"   # Fire Scar
BUFFER_layer = 'canyon2016_buffer' #Reference BUFFER AREA

Scar = gpd.read_file(project_db, layer=Scar_layer)
BUFFER = gpd.read_file(project_db, layer=BUFFER_layer)

In [24]:
meta = {
    # --- GRASS ---
    "GR1": ("Grass", "Arid–semiarid (EMC 15%)", "Short, patchy, possibly grazed; spread moderate; flame low."),
    "GR2": ("Grass", "Arid–semiarid (EMC 15%)", "Moderately coarse, ~1 ft; spread high; flame moderate."),
    "GR4": ("Grass", "Arid–semiarid (EMC 15%)", "Moderately coarse, ~2 ft; spread very high; flame high."),
    "GR7": ("Grass", "Arid–semiarid (EMC 15%)", "Moderately coarse, ~3 ft; spread very high; flame very high."),
    "GR3": ("Grass", "Subhumid–humid (EMC 30–40%)", "Very coarse, ~2 ft; spread high; flame moderate."),
    "GR5": ("Grass", "Subhumid–humid (EMC 30–40%)", "Dense, coarse, 1–2 ft; spread very high; flame high."),
    "GR6": ("Grass", "Subhumid–humid (EMC 30–40%)", "Dryland grass 1–2 ft; spread very high; flame very high."),
    "GR8": ("Grass", "Subhumid–humid (EMC 30–40%)", "Heavy, coarse, 3–5 ft; spread very high; flame very high."),
    "GR9": ("Grass", "Subhumid–humid (EMC 30–40%)", "Very heavy, coarse, 5–8 ft; spread extreme; flame extreme."),
    # --- GRASS-SHRUB ---
    "GS1": ("Grass-Shrub", "Arid–semiarid (EMC 15%)", "Shrubs ~1 ft, low grass; spread moderate; flame low."),
    "GS2": ("Grass-Shrub", "Arid–semiarid (EMC 15%)", "Shrubs 1–3 ft, moderate grass; spread high; flame moderate."),
    "GS3": ("Grass-Shrub", "Subhumid–humid (EMC 30–40%)", "Moderate grass/shrub <2 ft; spread high; flame moderate."),
    "GS4": ("Grass-Shrub", "Subhumid–humid (EMC 30–40%)", "Heavy grass/shrub >2 ft; spread high; flame very high."),
    # --- SHRUB ---
    "SH1": ("Shrub", "Arid–semiarid (EMC 15%)", "Low shrub load ~1 ft; spread very low; flame very low."),
    "SH2": ("Shrub", "Arid–semiarid (EMC 15%)", "Moderate load ~1 ft; no grass; spread low; flame low."),
    "SH5": ("Shrub", "Arid–semiarid (EMC 15%)", "Heavy shrubs 4–6 ft; spread very high; flame very high."),
    "SH7": ("Shrub", "Arid–semiarid (EMC 15%)", "Very heavy shrubs 4–6 ft; spread high; flame very high."),
    "SH3": ("Shrub", "Subhumid–humid (EMC 30–40%)", "Moderate shrubs (maybe pine/herb); 2–3 ft; spread low; flame low."),
    "SH4": ("Shrub", "Subhumid–humid (EMC 30–40%)", "Low–moderate shrubs/litter (~3 ft); spread high; flame moderate."),
    "SH6": ("Shrub", "Subhumid–humid (EMC 30–40%)", "Dense shrubs, little/no herb; ~2 ft; spread high; flame high."),
    "SH8": ("Shrub", "Subhumid–humid (EMC 30–40%)", "Dense shrubs, ~3 ft; spread high; flame high."),
    "SH9": ("Shrub", "Subhumid–humid (EMC 30–40%)", "Dense, fine-branched, 4–6 ft; spread high; flame very high."),
    # --- TIMBER-UNDERSTORY ---
    "TU1": ("Timber-Understory", "Semiarid–subhumid (EMC 20%)", "Low grass/shrub + litter; spread low; flame low."),
    "TU2": ("Timber-Understory", "Humid (EMC 30%)", "Moderate litter with shrubs; spread moderate; flame low."),
    "TU3": ("Timber-Understory", "Humid (EMC 30%)", "Moderate litter + grass/shrubs; spread high; flame moderate."),
    "TU4": ("Timber-Understory", "Semiarid–subhumid (EMC 20%)", "Short conifers w/ grass/moss; spread moderate; flame moderate."),
    "TU5": ("Timber-Understory", "Semiarid–subhumid (EMC 20%)", "High conifer litter + shrubs; spread moderate; flame moderate."),
    # --- TIMBER LITTER ---
    "TL1": ("Timber Litter", "Recently burned", "Light–moderate load, 1–2 in deep; spread very low; flame very low."),
    "TL2": ("Timber Litter", "Broadleaf litter", "Low load, compact; spread very low; flame very low."),
    "TL3": ("Timber Litter", "Other conifer litter", "Moderate conifer litter; spread very low; flame low."),
    "TL4": ("Timber Litter", "Mixed fine & coarse", "Moderate load incl. small logs; spread low; flame low."),
    "TL5": ("Timber Litter", "Conifer litter", "High load; light slash/mortality; spread low; flame low."),
    "TL6": ("Timber Litter", "Broadleaf litter", "Moderate load, less compact; spread moderate; flame low."),
    "TL7": ("Timber Litter", "Mixed fine & coarse", "Heavy load incl. larger logs; spread low; flame low."),
    "TL8": ("Timber Litter", "Long-needle pine", "Moderate load/compact; some herb; spread moderate; flame low."),
    "TL9": ("Timber Litter", "Broadleaf / needle drape", "Very high load; spread moderate; flame moderate."),
    # --- SLASH/BLOWDOWN ---
    "SB1": ("Slash-Blowdown", "Activity fuel", "10–20 t/ac; fuels 1–3 in; <1 ft depth; spread moderate; flame low."),
    "SB2": ("Slash-Blowdown", "Activity fuel", "7–12 t/ac; even 0–3 in; ~1 ft; spread moderate; flame moderate."),
    "SB3": ("Slash-Blowdown", "Activity fuel", "7–12 t/ac; weighted to <0.25 in; >1 ft; spread high; flame high."),
    # blowdown variants share codes with SB2/SB3/SB4 as behavior descriptors
    "SB4": ("Slash-Blowdown", "Blowdown (total)", "Total blowdown; not compacted; foliage attached; spread very high; flame very high."),
    # --- NONBURNABLE ---
    "NB1": ("Nonburnable", "—", "Urban/suburban; insufficient wildland fuel."),
    "NB2": ("Nonburnable", "—", "Snow/ice."),
    "NB3": ("Nonburnable", "—", "Agricultural field maintained nonburnable."),
    "NB8": ("Nonburnable", "—", "Open water."),
    "NB9": ("Nonburnable", "—", "Bare ground."),
}

In [25]:
# Track orders (within *same* general_type + climate)
ORDER = {
    'Shrub': {
        'Arid–semiarid (EMC 15%)':       ['SH1','SH2','SH5','SH7'],
        'Subhumid–humid (EMC 30–40%)':   ['SH3','SH4','SH6','SH8','SH9'],
    },
    'Grass': {
        'Arid–semiarid (EMC 15%)':       ['GR1','GR2','GR4','GR7'],
        'Subhumid–humid (EMC 30–40%)':   ['GR3','GR5','GR6','GR8','GR9'],
    },
    'Grass-Shrub': {
        'Arid–semiarid (EMC 15%)':       ['GS1','GS2'],
        'Subhumid–humid (EMC 30–40%)':   ['GS3','GS4'],
    },
    'Timber-Understory': {
        'Semiarid–subhumid (EMC 20%)':   ['TU1','TU4','TU5'],
        'Humid (EMC 30%)':               ['TU2','TU3'],
    },
}

In [26]:
fbfm40_meta = (
    pd.DataFrame.from_dict(meta, orient="index", columns=["general_type", "climate", "summary"])
      .reset_index()
      .rename(columns={"index": "MODEL"})
)

In [27]:
# 1) Read DBF to DataFrame
tbl = DBF(dbf_path, load=True, char_decode_errors='ignore')
lut_df = pd.DataFrame(iter(tbl))
value_col = 'VALUE'
model_col = "FBFM40"
# Keep just code + label; standardize names
lut_df = lut_df[[value_col, model_col]].copy()
lut_df.columns = ["VALUE", "FBFM40"]
lut_df = lut_df.rename(columns={"FBFM40": "MODEL"})

In [28]:
# -----------------------
# Run zonal statistics
# -----------------------
zs = zonal_stats(
    BUFFER,
    raster_path,
    categorical=True,   # return pixel counts per EVT code
    nodata=-9999)

# 3) Build long table from your zs (fid, VALUE, pixels)
rows = []
for fid, counts in enumerate(zs):
    for val, cnt in counts.items():
        rows.append({"fid": fid, "VALUE": int(val), "pixels": int(cnt)})
df_counts = pd.DataFrame(rows)
# 4) Join codes → names
df_counts = df_counts.merge(lut_df, on="VALUE", how="left")
df_counts = df_counts.merge(fbfm40_meta, on="MODEL", how="left")
df_counts

,fid,VALUE,pixels,MODEL,general_type,climate,summary
0,0,91,5191,NB1,Nonburnable,—,Urban/suburban; insufficient wildland fuel.
1,0,98,5618,NB8,Nonburnable,—,Open water.
2,0,99,1109,NB9,Nonburnable,—,Bare ground.
3,0,101,473,GR1,Grass,Arid–semiarid (EMC 15%),"Short, patchy, possibly grazed; spread moderat..."
4,0,102,13570,GR2,Grass,Arid–semiarid (EMC 15%),"Moderately coarse, ~1 ft; spread high; flame m..."
5,0,103,36,GR3,Grass,Subhumid–humid (EMC 30–40%),"Very coarse, ~2 ft; spread high; flame moderate."
6,0,121,1789,GS1,Grass-Shrub,Arid–semiarid (EMC 15%),"Shrubs ~1 ft, low grass; spread moderate; flam..."
7,0,122,31189,GS2,Grass-Shrub,Arid–semiarid (EMC 15%),"Shrubs 1–3 ft, moderate grass; spread high; fl..."
8,0,123,12151,GS3,Grass-Shrub,Subhumid–humid (EMC 30–40%),Moderate grass/shrub <2 ft; spread high; flame...
9,0,142,8568,SH2,Shrub,Arid–semiarid (EMC 15%),Moderate load ~1 ft; no grass; spread low; fla...


In [29]:
# -----------------------
# Run zonal statistics
# -----------------------
zs_lf = zonal_stats(
    Scar,
    raster_path,
    categorical=True,   # return pixel counts per EVT code
    nodata=-9999)

# 3) Build long table from  zs_latest_fire (fid, VALUE, pixels)
rows = []
for fid, counts in enumerate(zs_lf):
    for val, cnt in counts.items():
        rows.append({"fid": fid, "VALUE": int(val), "pixels": int(cnt)})
df_counts_lf = pd.DataFrame(rows)
# 4) Join codes → names
df_counts_lf = df_counts_lf.merge(lut_df, on="VALUE", how="left")
df_counts_lf = df_counts_lf.merge(fbfm40_meta, on="MODEL", how="left")
df_counts_lf

,fid,VALUE,pixels,MODEL,general_type,climate,summary
0,0,91,2428,NB1,Nonburnable,—,Urban/suburban; insufficient wildland fuel.
1,0,99,213,NB9,Nonburnable,—,Bare ground.
2,0,101,429,GR1,Grass,Arid–semiarid (EMC 15%),"Short, patchy, possibly grazed; spread moderat..."
3,0,102,6371,GR2,Grass,Arid–semiarid (EMC 15%),"Moderately coarse, ~1 ft; spread high; flame m..."
4,0,121,1442,GS1,Grass-Shrub,Arid–semiarid (EMC 15%),"Shrubs ~1 ft, low grass; spread moderate; flam..."
5,0,122,23113,GS2,Grass-Shrub,Arid–semiarid (EMC 15%),"Shrubs 1–3 ft, moderate grass; spread high; fl..."
6,0,123,97,GS3,Grass-Shrub,Subhumid–humid (EMC 30–40%),Moderate grass/shrub <2 ft; spread high; flame...
7,0,142,8523,SH2,Shrub,Arid–semiarid (EMC 15%),Moderate load ~1 ft; no grass; spread low; fla...
8,0,144,60,SH4,Shrub,Subhumid–humid (EMC 30–40%),Low–moderate shrubs/litter (~3 ft); spread hig...
9,0,145,224,SH5,Shrub,Arid–semiarid (EMC 15%),Heavy shrubs 4–6 ft; spread very high; flame v...


In [30]:
# Modal (most pixels) MODEL in the ring, by general_type
ring_type_modal = (
    df_counts.groupby(['general_type','MODEL'], dropna=False)['pixels']
             .sum().reset_index()
)
ring_type_modal = ring_type_modal.sort_values(['general_type','pixels'], ascending=[True, False])
ring_type_modal = (ring_type_modal.groupby('general_type', as_index=False)
                                  .first()[['general_type','MODEL']]
                                  .rename(columns={'MODEL':'MODEL_modal'}))

print("Ring modal by type:")
print(ring_type_modal)

Ring modal by type:
        general_type MODEL_modal
0              Grass         GR2
1        Grass-Shrub         GS2
2        Nonburnable         NB8
3              Shrub         SH2
4     Slash-Blowdown         SB2
5      Timber Litter         TL6
6  Timber-Understory         TU5


In [31]:
def model_to_preferred_value(df_counts):
    ref = df_counts.copy()
    mv = (
        ref.groupby(["MODEL","VALUE"], dropna=False)["pixels"]
           .sum()
           .reset_index()
    )
    idx = mv.groupby("MODEL")["pixels"].idxmax()
    best = mv.loc[idx, ["MODEL","VALUE"]].dropna()
    return dict(zip(best["MODEL"], best["VALUE"]))


In [32]:
def no_downgrade(row):
    gt = row.get('general_type')
    cl = row.get('climate')
    before = row.get('MODEL_before')
    after  = row.get('MODEL_after')
    order = ORDER.get(gt, {}).get(cl)
    if order and (before in order) and (after in order):
        # if ring modal ranks *below* original, keep original
        return after if order.index(after) >= order.index(before) else before
    return after

In [33]:
""" # Canonical VALUE per MODEL from the ring (dominant VALUE)
model_value_map_ring = model_to_preferred_value(df_counts)  # uses your helper

# Build upgraded table inside the scar
upgraded_df_lf = df_counts_lf.copy()
upgraded_df_lf = upgraded_df_lf.rename(columns={'MODEL':'MODEL_before', 'VALUE':'VALUE_before'})

# Join the ring’s modal MODEL for the same general_type
upgraded_df_lf = upgraded_df_lf.merge(ring_type_modal, on='general_type', how='left')

# If a general_type isn’t present in the ring, keep original
upgraded_df_lf['MODEL_after'] = upgraded_df_lf['MODEL_modal'].fillna(upgraded_df_lf['MODEL_before'])
upgraded_df_lf.drop(columns=['MODEL_modal'], inplace=True)

# VALUE_after from the ring’s canonical VALUE for that MODEL; fallback to VALUE_before
upgraded_df_lf['VALUE_after'] = upgraded_df_lf['MODEL_after'].map(model_value_map_ring).fillna(upgraded_df_lf['VALUE_before'])

# Nice column order
front = ['fid','MODEL_before','VALUE_before','MODEL_after','VALUE_after','general_type','climate','summary','pixels']
upgraded_df_lf = upgraded_df_lf[[c for c in front if c in upgraded_df_lf.columns] + 
                                [c for c in upgraded_df_lf.columns if c not in front]]

# Quick sanity checks
print("Totals equal?",
      upgraded_df_lf['pixels'].sum() ==
      upgraded_df_lf.groupby('MODEL_after')['pixels'].sum().sum())

# What changed?
changes = (upgraded_df_lf.loc[upgraded_df_lf['MODEL_before'] != upgraded_df_lf['MODEL_after'],
                              ['MODEL_before','VALUE_before','MODEL_after','VALUE_after','general_type','pixels']]
           .sort_values(['general_type','pixels'], ascending=[True, False]))
print("Sample changes:")
print(changes.head(15)) """


' # Canonical VALUE per MODEL from the ring (dominant VALUE)\nmodel_value_map_ring = model_to_preferred_value(df_counts)  # uses your helper\n\n# Build upgraded table inside the scar\nupgraded_df_lf = df_counts_lf.copy()\nupgraded_df_lf = upgraded_df_lf.rename(columns={\'MODEL\':\'MODEL_before\', \'VALUE\':\'VALUE_before\'})\n\n# Join the ring’s modal MODEL for the same general_type\nupgraded_df_lf = upgraded_df_lf.merge(ring_type_modal, on=\'general_type\', how=\'left\')\n\n# If a general_type isn’t present in the ring, keep original\nupgraded_df_lf[\'MODEL_after\'] = upgraded_df_lf[\'MODEL_modal\'].fillna(upgraded_df_lf[\'MODEL_before\'])\nupgraded_df_lf.drop(columns=[\'MODEL_modal\'], inplace=True)\n\n# VALUE_after from the ring’s canonical VALUE for that MODEL; fallback to VALUE_before\nupgraded_df_lf[\'VALUE_after\'] = upgraded_df_lf[\'MODEL_after\'].map(model_value_map_ring).fillna(upgraded_df_lf[\'VALUE_before\'])\n\n# Nice column order\nfront = [\'fid\',\'MODEL_before\',\'VALUE

In [34]:
# ---- 1) Modal per (general_type, climate) in the ring ----
agg_tc = (df_counts
          .dropna(subset=['general_type','climate','MODEL'])
          .groupby(['general_type','climate','MODEL'], dropna=False)['pixels']
          .sum().reset_index())

idx_tc = agg_tc.groupby(['general_type','climate'])['pixels'].idxmax()
ring_type_climate_modal = (agg_tc.loc[idx_tc, ['general_type','climate','MODEL']]
                           .rename(columns={'MODEL':'MODEL_modal_tc'}))

# Fallback: modal per general_type only (in case a climate subtype is absent in ring)
agg_t = (df_counts
         .dropna(subset=['general_type','MODEL'])
         .groupby(['general_type','MODEL'], dropna=False)['pixels']
         .sum().reset_index())
idx_t = agg_t.groupby('general_type')['pixels'].idxmax()
ring_type_modal = (agg_t.loc[idx_t, ['general_type','MODEL']]
                   .rename(columns={'MODEL':'MODEL_modal_type'}))

# ---- 2) Apply to the burned table (inside) ----
upgraded_df_lf = df_counts_lf.copy()
upgraded_df_lf = upgraded_df_lf.rename(columns={'MODEL':'MODEL_before', 'VALUE':'VALUE_before'})

# Join climate-constrained modal first; then fallback to type modal
upgraded_df_lf = upgraded_df_lf.merge(ring_type_climate_modal, on=['general_type','climate'], how='left')
upgraded_df_lf = upgraded_df_lf.merge(ring_type_modal, on='general_type', how='left')

# Start with no change by default
upgraded_df_lf['MODEL_after'] = upgraded_df_lf['MODEL_before']

# Masks by family
mask_nb = upgraded_df_lf['MODEL_before'].astype(str).str.upper().str.startswith('NB') | \
          upgraded_df_lf.get('general_type', pd.Series(False)).eq('Nonburnable')
mask_tl = upgraded_df_lf['general_type'].eq('Timber Litter')
mask_gr = upgraded_df_lf['general_type'].eq('Grass')
mask_gs = upgraded_df_lf['general_type'].eq('Grass-Shrub')
mask_tu = upgraded_df_lf['general_type'].eq('Timber-Understory')
mask_sh = upgraded_df_lf['general_type'].eq('Shrub')
mask_other = ~(mask_nb | mask_tl | mask_gr | mask_gs | mask_tu | mask_sh)

# 1) NBx: freeze
upgraded_df_lf.loc[mask_nb, 'MODEL_after'] = upgraded_df_lf.loc[mask_nb, 'MODEL_before']

# 2) TL: keep as-is (no remap)
upgraded_df_lf.loc[mask_tl, 'MODEL_after'] = upgraded_df_lf.loc[mask_tl, 'MODEL_before']

# 3) GR & GS: require type+climate; if missing in ring, keep original (no type-only fallback)
upgraded_df_lf.loc[mask_gr | mask_gs, 'MODEL_after'] = (
    upgraded_df_lf.loc[mask_gr | mask_gs, 'MODEL_modal_tc']
        .fillna(upgraded_df_lf.loc[mask_gr | mask_gs, 'MODEL_before'])
)

# 4) TU: require same-climate modal; if missing, keep original
#     => TU1 will map to TU4/TU5 if those are modal in the semiarid–subhumid ring;
#        it will NOT cross to humid TU2 unless your ring modal_tc is actually humid for that pixel.
upgraded_df_lf.loc[mask_tu, 'MODEL_after'] = (
    upgraded_df_lf.loc[mask_tu, 'MODEL_modal_tc']
        .fillna(upgraded_df_lf.loc[mask_tu, 'MODEL_before'])
)

# 5) SH (and others): prefer type+climate, else type-only, else keep
upgraded_df_lf.loc[mask_sh | mask_other, 'MODEL_after'] = (
    upgraded_df_lf.loc[mask_sh | mask_other, 'MODEL_modal_tc']
        .fillna(upgraded_df_lf.loc[mask_sh | mask_other, 'MODEL_modal_type'])
        .fillna(upgraded_df_lf.loc[mask_sh | mask_other, 'MODEL_before'])
)

# Clean up helper columns if present
upgraded_df_lf.drop(columns=['MODEL_modal_tc','MODEL_modal_type'], inplace=True, errors='ignore')

# Apply to families with track definitions (SH, GR, GS, TU)
mask_track = upgraded_df_lf['general_type'].isin(ORDER.keys())
upgraded_df_lf.loc[mask_track, 'MODEL_after'] = (
    upgraded_df_lf.loc[mask_track].apply(no_downgrade, axis=1)
)

# VALUE_after from the ring’s canonical VALUE for that model (dominant VALUE in ring)
model_value_map_ring = model_to_preferred_value(df_counts)
upgraded_df_lf['VALUE_after'] = (
    upgraded_df_lf['MODEL_after'].map(model_value_map_ring)
    .fillna(upgraded_df_lf['VALUE_before'])
)

# Extra NB safety (keeps NB VALUE identical)
upgraded_df_lf.loc[mask_nb, 'VALUE_after'] = upgraded_df_lf.loc[mask_nb, 'VALUE_before']
upgraded_df_lf['Sample Area']= 'Alisal Fire 2021'


In [35]:
#Quick sanity checks
print("Totals equal?",
      upgraded_df_lf['pixels'].sum() ==
      upgraded_df_lf.groupby('MODEL_after')['pixels'].sum().sum())

# What changed?
changes = (upgraded_df_lf.loc[upgraded_df_lf['MODEL_before'] != upgraded_df_lf['MODEL_after'],
                              ['MODEL_before','VALUE_before','MODEL_after','VALUE_after','general_type','pixels']]
           .sort_values(['general_type','pixels'], ascending=[True, False]))
print("Sample changes:")
print(changes.head(15)) 

Totals equal? True
Sample changes:
  MODEL_before  VALUE_before MODEL_after  VALUE_after general_type  pixels
2          GR1           101         GR2          102        Grass     429
4          GS1           121         GS2          122  Grass-Shrub    1442


In [36]:
upgraded_df_lf

,fid,VALUE_before,pixels,MODEL_before,general_type,climate,summary,MODEL_after,VALUE_after,Sample Area
0,0,91,2428,NB1,Nonburnable,—,Urban/suburban; insufficient wildland fuel.,NB1,91,Alisal Fire 2021
1,0,99,213,NB9,Nonburnable,—,Bare ground.,NB9,99,Alisal Fire 2021
2,0,101,429,GR1,Grass,Arid–semiarid (EMC 15%),"Short, patchy, possibly grazed; spread moderat...",GR2,102,Alisal Fire 2021
3,0,102,6371,GR2,Grass,Arid–semiarid (EMC 15%),"Moderately coarse, ~1 ft; spread high; flame m...",GR2,102,Alisal Fire 2021
4,0,121,1442,GS1,Grass-Shrub,Arid–semiarid (EMC 15%),"Shrubs ~1 ft, low grass; spread moderate; flam...",GS2,122,Alisal Fire 2021
5,0,122,23113,GS2,Grass-Shrub,Arid–semiarid (EMC 15%),"Shrubs 1–3 ft, moderate grass; spread high; fl...",GS2,122,Alisal Fire 2021
6,0,123,97,GS3,Grass-Shrub,Subhumid–humid (EMC 30–40%),Moderate grass/shrub <2 ft; spread high; flame...,GS3,123,Alisal Fire 2021
7,0,142,8523,SH2,Shrub,Arid–semiarid (EMC 15%),Moderate load ~1 ft; no grass; spread low; fla...,SH2,142,Alisal Fire 2021
8,0,144,60,SH4,Shrub,Subhumid–humid (EMC 30–40%),Low–moderate shrubs/litter (~3 ft); spread hig...,SH4,144,Alisal Fire 2021
9,0,145,224,SH5,Shrub,Arid–semiarid (EMC 15%),Heavy shrubs 4–6 ft; spread very high; flame v...,SH5,145,Alisal Fire 2021


In [37]:
# (Optional) tidy columns
front = ['fid','MODEL_before','VALUE_before','MODEL_after','VALUE_after','general_type','climate','summary','pixels']
upgraded_df_lf = upgraded_df_lf[[c for c in front if c in upgraded_df_lf.columns] +
                                [c for c in upgraded_df_lf.columns if c not in front]]

In [38]:
upgraded_df_lf

,fid,MODEL_before,VALUE_before,MODEL_after,VALUE_after,general_type,climate,summary,pixels,Sample Area
0,0,NB1,91,NB1,91,Nonburnable,—,Urban/suburban; insufficient wildland fuel.,2428,Alisal Fire 2021
1,0,NB9,99,NB9,99,Nonburnable,—,Bare ground.,213,Alisal Fire 2021
2,0,GR1,101,GR2,102,Grass,Arid–semiarid (EMC 15%),"Short, patchy, possibly grazed; spread moderat...",429,Alisal Fire 2021
3,0,GR2,102,GR2,102,Grass,Arid–semiarid (EMC 15%),"Moderately coarse, ~1 ft; spread high; flame m...",6371,Alisal Fire 2021
4,0,GS1,121,GS2,122,Grass-Shrub,Arid–semiarid (EMC 15%),"Shrubs ~1 ft, low grass; spread moderate; flam...",1442,Alisal Fire 2021
5,0,GS2,122,GS2,122,Grass-Shrub,Arid–semiarid (EMC 15%),"Shrubs 1–3 ft, moderate grass; spread high; fl...",23113,Alisal Fire 2021
6,0,GS3,123,GS3,123,Grass-Shrub,Subhumid–humid (EMC 30–40%),Moderate grass/shrub <2 ft; spread high; flame...,97,Alisal Fire 2021
7,0,SH2,142,SH2,142,Shrub,Arid–semiarid (EMC 15%),Moderate load ~1 ft; no grass; spread low; fla...,8523,Alisal Fire 2021
8,0,SH4,144,SH4,144,Shrub,Subhumid–humid (EMC 30–40%),Low–moderate shrubs/litter (~3 ft); spread hig...,60,Alisal Fire 2021
9,0,SH5,145,SH5,145,Shrub,Arid–semiarid (EMC 15%),Heavy shrubs 4–6 ft; spread very high; flame v...,224,Alisal Fire 2021


In [39]:
# --- safety: freeze NBx and TL (no change) ---
mask_nb = upgraded_df_lf['MODEL_before'].astype(str).str.upper().str.startswith('NB') \
          | upgraded_df_lf['general_type'].eq('Nonburnable')
mask_tl = upgraded_df_lf['general_type'].eq('Timber Litter')
upgraded_df_lf.loc[mask_nb | mask_tl, ['MODEL_after','VALUE_after']] = \
    upgraded_df_lf.loc[mask_nb | mask_tl, ['MODEL_before','VALUE_before']].values

# Labels to match Marc’s “Vegetation Type” wording (edit if you want different names)
veg_label_map = {
    'Shrub': 'Chaparral Shrubland',
    'Timber-Understory': 'Woodland (Drainages)',
    'Grass': 'Grassland',
    # keep others as-is unless you want custom labels:
    'Grass-Shrub': 'Grass–Shrub',
    'Timber Litter': 'Timber Litter',
    'Slash-Blowdown': 'Slash/Blowdown',
    'Nonburnable': 'Nonburnable',
}

reference_label = 'Adjacent unburned 1 km buffer (>20 yr)'

# --- paste-ready table (one row per inside class) ---
sheet_df = upgraded_df_lf.copy()
sheet_df['Vegetation Type']       = sheet_df['general_type'].map(veg_label_map).fillna(sheet_df['general_type'])
sheet_df['Sample Number']         = sheet_df['pixels'].astype(int)   # per your note: use pixels as the sample number
sheet_df['Sample Area (scar)']    = sheet_df.get('Sample Area', 'Alisal Fire 2021')
sheet_df['LF24 model (scar)']     = sheet_df['MODEL_before']
sheet_df['Raster code (scar)']    = sheet_df['VALUE_before'].astype('Int64')
sheet_df['Reference Sample Area'] = reference_label
sheet_df['LF24 model (ref)']      = sheet_df['MODEL_after']
sheet_df['Raster code (ref)']     = sheet_df['VALUE_after'].astype('Int64')

# keep only the columns in Marc’s layout order (skip the visual blank divider column)
sheet_df = sheet_df[[
    'Vegetation Type',
    'Sample Number',
    'Sample Area (scar)',
    'LF24 model (scar)',
    'Raster code (scar)',
    'Reference Sample Area',
    'LF24 model (ref)',
    'Raster code (ref)',
]].sort_values(['Vegetation Type','LF24 model (scar)']).reset_index(drop=True)



In [40]:
sheet_df

,Vegetation Type,Sample Number,Sample Area (scar),LF24 model (scar),Raster code (scar),Reference Sample Area,LF24 model (ref),Raster code (ref)
0,Chaparral Shrubland,8523,Alisal Fire 2021,SH2,142,Adjacent unburned 1 km buffer (>20 yr),SH2,142
1,Chaparral Shrubland,60,Alisal Fire 2021,SH4,144,Adjacent unburned 1 km buffer (>20 yr),SH4,144
2,Chaparral Shrubland,224,Alisal Fire 2021,SH5,145,Adjacent unburned 1 km buffer (>20 yr),SH5,145
3,Grassland,429,Alisal Fire 2021,GR1,101,Adjacent unburned 1 km buffer (>20 yr),GR2,102
4,Grassland,6371,Alisal Fire 2021,GR2,102,Adjacent unburned 1 km buffer (>20 yr),GR2,102
5,Grass–Shrub,1442,Alisal Fire 2021,GS1,121,Adjacent unburned 1 km buffer (>20 yr),GS2,122
6,Grass–Shrub,23113,Alisal Fire 2021,GS2,122,Adjacent unburned 1 km buffer (>20 yr),GS2,122
7,Grass–Shrub,97,Alisal Fire 2021,GS3,123,Adjacent unburned 1 km buffer (>20 yr),GS3,123
8,Nonburnable,2428,Alisal Fire 2021,NB1,91,Adjacent unburned 1 km buffer (>20 yr),NB1,91
9,Nonburnable,213,Alisal Fire 2021,NB9,99,Adjacent unburned 1 km buffer (>20 yr),NB9,99


In [41]:
pairs = (upgraded_df_lf[['VALUE_before','VALUE_after','pixels']]
         .groupby(['VALUE_before','VALUE_after'])['pixels'].sum().reset_index())
idx = pairs.groupby('VALUE_before')['pixels'].idxmax()
reclass_df = (pairs.loc[idx, ['VALUE_before','VALUE_after']]
              .astype(int).sort_values('VALUE_before').reset_index(drop=True))
# reclass_df.to_csv(r"C:\path\to\remap_pairs.csv", index=False)


In [42]:
reclass_df

,VALUE_before,VALUE_after
0,91,91
1,99,99
2,101,102
3,102,102
4,121,122
5,122,122
6,123,123
7,142,142
8,144,144
9,145,145
